## TOC
* [Target Dependence](#target-dependence)
* [Data Preprocessing](#data-preprocessing)
* [First Bullet Header](#first-bullet)
* [Fit the Model](#fit-model)
* [Single Target Classifier](#single-class)

## First Bullet Header <a class="anchor" id="first-bullet"></a>

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd

project_root = Path().absolute()
data_folder = project_root / 'data'
data_file_path = data_folder / 'conneticut_wide_form.csv'
data = pd.read_csv(data_file_path)

data['date'] = pd.to_datetime(data['date'])
data['age'] = pd.to_numeric(data['age'])

/tmp/ipykernel_102629/1261883196.py:8: DtypeWarning: Columns (25,35,40,42,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_file_path)


In [2]:
def print_missing_values_percent(data):
    data_missing_values = data.isnull().sum()
    missing_values_percentage = (data_missing_values / len(data)) * 100
    missing_values_percentage.sort_values(inplace=True)

    for index, value in missing_values_percentage.items():
        print(str(index) + ': ' + str(np.round(value, decimals=2)) + '%')

In [3]:
print_missing_values_percent(data)

CaseIdentifier: 0.0%
__id: 0.0%
date: 0.0%
datetype: 0.0%
cod: 0.0%
death_day: 0.0%
death_month: 0.0%
death_month_num: 0.0%
death_day_is_weekend: 0.0%
death_day_of_week: 0.0%
death_year: 0.0%
death_day_week_of_year: 0.0%
deathcitygeo: 0.01%
deathcitygeo_state: 0.01%
age: 0.02%
mannerofdeath: 0.08%
sex: 0.08%
race: 0.48%
residencecitygeo: 1.39%
residencecitygeo_state: 1.39%
injurycity: 1.49%
injurycitygeo: 2.15%
injurycitygeo_state: 2.15%
cod_matched: 2.95%
drug_meta: 2.96%
injuryplace: 2.99%
injurycitygeo_city: 3.63%
residencecity: 4.97%
residencecitygeo_city: 6.37%
descriptionofinjury: 6.74%
opiate_meta: 10.0%
residencecounty: 10.52%
location: 11.26%
residencestate: 16.59%
deathcity: 23.24%
deathcitygeo_city: 23.25%
injurystate: 25.28%
anyopioid: 25.32%
injurycounty: 27.83%
deathcounty: 32.48%
fentanyl: 32.82%
fentanyl_meta: 33.34%
fentanyl.1: 33.45%
death_state: 42.63%
nonfentanyl_opioid_meta: 51.69%
stimulant_meta: 60.49%
cocaine: 61.79%
cocaine.1: 65.23%
heroin: 70.14%
sedative_met

In [4]:
# cocaine.1 is the first meta_drug column added via pipeline
# comparison of cocaine (official) vs cocaine.1 (meta) shows similar missing entries, so the meta columns will be dropped
first_column_to_drop_index = data.columns.get_loc('cocaine.1')
data_no_meta_columns = data.iloc[:, :first_column_to_drop_index]

## Are Target Variables Dependent? <a class="anchor" id="target-dependence"></a>

In [5]:
# See what target labels are most common.
# For the present data set, this is the first column. In the future a more programmatic way would be helpful to find the first column.
# I choose not to use the meta columns fo simplicity.

# first_drug_column = 'heroin' 
# first_drug_column_index = data.columns.get_loc(first_drug_column)
# last_drug_column = ''
# last_drug_column_index = data.columns.get_loc(last_drug_column)
# target_df = data.iloc[:, first_drug_column_index:last_drug_column_index]
# print_missing_values_percent(target_df)

In [6]:
# target_df = target_df.fillna(0)

In [7]:
# see if target labels are correlated to determine what model to use

# target_correlation_matrix = target_df.corr()
# import seaborn as sns
# import matplotlib.pyplot as plt
# sns.heatmap(target_correlation_matrix, annot=False, cmap="coolwarm")
# plt.show()

In [8]:
# high_correlation_threshold = 0.5
# high_correlation_pairs = target_correlation_matrix[(target_correlation_matrix > high_correlation_threshold) | (target_correlation_matrix < -high_correlation_threshold)].stack()
# high_correlation_df = high_correlation_pairs.reset_index()
# high_correlation_df.columns = ['variable1', 'variable2', 'correlation']
# high_correlation_df = high_correlation_df[high_correlation_df['variable1'] != high_correlation_df['variable2']]
# print(high_correlation_df)

The target features that have strong correlation are to be expected. For example, cocaine and stimulant_meta are expected to be correlated because cocaine is a stimulant.

In [9]:
# pick the top 3 most common causes of death that aren't a meta/general category (e.g. stimulant_meta)

# top3_target_df = target_df[['cocaine.1', 'heroin.1', 'fentanyl.1']]
# top3_target_correlation = top3_target_df.corr()
# top3_target_correlation

## Data Preprocessing <a class="anchor" id="data-preprocessing"></a>

In [10]:
# features_df = data.iloc[:, :first_drug_column_index]
print_missing_values_percent(data_no_meta_columns)

CaseIdentifier: 0.0%
__id: 0.0%
date: 0.0%
datetype: 0.0%
cod: 0.0%
death_day: 0.0%
death_month: 0.0%
death_month_num: 0.0%
death_day_is_weekend: 0.0%
death_day_of_week: 0.0%
death_year: 0.0%
death_day_week_of_year: 0.0%
deathcitygeo: 0.01%
deathcitygeo_state: 0.01%
age: 0.02%
mannerofdeath: 0.08%
sex: 0.08%
race: 0.48%
residencecitygeo: 1.39%
residencecitygeo_state: 1.39%
injurycity: 1.49%
injurycitygeo: 2.15%
injurycitygeo_state: 2.15%
injuryplace: 2.99%
injurycitygeo_city: 3.63%
residencecity: 4.97%
residencecitygeo_city: 6.37%
descriptionofinjury: 6.74%
residencecounty: 10.52%
location: 11.26%
residencestate: 16.59%
deathcity: 23.24%
deathcitygeo_city: 23.25%
injurystate: 25.28%
anyopioid: 25.32%
injurycounty: 27.83%
deathcounty: 32.48%
fentanyl: 32.82%
death_state: 42.63%
cocaine: 61.79%
heroin: 70.14%
ethanol: 73.28%
benzodiazepine: 77.32%
ethnicity: 78.59%
heroin_morph_codeine: 81.62%
othersignifican: 89.99%
locationifother: 90.03%
xylazine: 91.0%
methadone: 91.0%
oxycodone: 91.

Since a small (less than 1%) amount of data is missing either age or sex, just drop the entries missing these variables. Also, drop any columns that are completely empty.

In [11]:
data_no_meta_columns = data_no_meta_columns.dropna(subset=['sex', 'race'])
data_no_meta_columns = data_no_meta_columns.dropna(axis=1, how='all')

In [12]:
print_missing_values_percent(data_no_meta_columns)

CaseIdentifier: 0.0%
__id: 0.0%
date: 0.0%
datetype: 0.0%
sex: 0.0%
race: 0.0%
cod: 0.0%
death_day_of_week: 0.0%
death_month: 0.0%
death_year: 0.0%
death_day_is_weekend: 0.0%
death_day_week_of_year: 0.0%
death_day: 0.0%
death_month_num: 0.0%
deathcitygeo_state: 0.01%
age: 0.01%
deathcitygeo: 0.01%
mannerofdeath: 0.08%
residencecitygeo_state: 1.4%
residencecitygeo: 1.4%
injurycity: 1.49%
injurycitygeo: 2.13%
injurycitygeo_state: 2.13%
injuryplace: 3.0%
injurycitygeo_city: 3.62%
residencecity: 4.85%
residencecitygeo_city: 6.25%
descriptionofinjury: 6.77%
residencecounty: 10.39%
location: 11.08%
residencestate: 16.5%
deathcity: 23.06%
deathcitygeo_city: 23.07%
injurystate: 25.34%
anyopioid: 25.38%
injurycounty: 27.9%
deathcounty: 32.32%
fentanyl: 32.85%
death_state: 42.52%
cocaine: 61.89%
heroin: 70.03%
ethanol: 73.28%
benzodiazepine: 77.32%
ethnicity: 78.6%
heroin_morph_codeine: 81.56%
othersignifican: 90.01%
locationifother: 90.03%
methadone: 91.0%
xylazine: 91.03%
oxycodone: 91.5%
fent

Clean up the 'sex' feature values next.

In [13]:
print(f"unique values for 'sex' column original: {data_no_meta_columns['sex'].unique()}")
data_no_meta_columns = data_no_meta_columns[~data_no_meta_columns['sex'].isin(['Unknown', 'X'])]
data_no_meta_columns['sex_encoded'] = data_no_meta_columns['sex'].map({'Male': 0, 'Female': 1})
columns_received_encoding = ['sex']

unique values for 'sex' column original: ['Male' 'Female' 'Unknown' 'X']


In [14]:
non_empty_counts_df = data_no_meta_columns.count()
non_empty_counts_df = non_empty_counts_df.reset_index()
non_empty_counts_df.to_csv('non_empty_counts.csv', index=False)

In [15]:
cod_counts = data_no_meta_columns['cod'].value_counts()
cod_counts.shape

(7593,)

There are 7000+ unique causes of death, for approximately 12,000 data points. So my plan to use this as a categorical variable will not happen because there are almost as many categories as observations (same order of magnitude)

Use geojson to group observations into Connecticut counties. Although this data already has columns for county (injurycounty and deathcounty), these columns are not well populated. So we take advantage of the fact that injurycitygeo and deathcitygeo are reliably populated and do a spatial join with the known geographical boundaries of the counties of Connecticut.

From examination, we see that we should use injurycitygeo over deathcitygeo because for recent years of deathcitygeo, observations were all assigned the same value. This is likely error in how the data was entered.

In [16]:
locations_year_df = data_no_meta_columns[['death_year','injurycitygeo', 'deathcitygeo']]
yearly_injurycitygeo_counts = locations_year_df.groupby('death_year')['injurycitygeo'].nunique()
print(f"unique geo locations for injury city by year: {yearly_injurycitygeo_counts}")
yearly_deathcitygeo_counts = locations_year_df.groupby('death_year')['deathcitygeo'].nunique()
print(f"\nunique geo locations for death city by year: {yearly_deathcitygeo_counts}")

unique geo locations for injury city by year: death_year
2012    106
2013    118
2014    136
2015    141
2016    144
2017    156
2018    154
2019    164
2020    143
2021    164
2022    153
2023    142
Name: injurycitygeo, dtype: int64

unique geo locations for death city by year: death_year
2012     92
2013     93
2014    116
2015    131
2016    129
2017    141
2018    140
2019    148
2020    132
2021    151
2022      1
2023      1
Name: deathcitygeo, dtype: int64


In [17]:
import geopandas as gpd
from shapely.geometry import shape
import json

counties_geojson_file_path = data_folder / 'CT_Counties.geojson'
geolocation_column_name = 'injurycitygeo'

data_no_meta_columns = data_no_meta_columns.dropna(subset=geolocation_column_name)
data_no_meta_columns.loc[:, geolocation_column_name] = data_no_meta_columns[geolocation_column_name].str.replace("'", '"')
data_no_meta_columns['geometry'] = data_no_meta_columns[geolocation_column_name].apply(lambda x: shape(json.loads(x)))
death_geodata = gpd.GeoDataFrame(data_no_meta_columns, geometry='geometry', crs=4326)

# CT counties geodata
with open(counties_geojson_file_path, 'r') as f:
    counties_geojson = json.load(f)
counties_gdf = gpd.read_file(counties_geojson_file_path)

county_geometry_gdf = counties_gdf[['COUNTY', 'geometry']]

joined_data_county_gdf = gpd.sjoin(death_geodata, county_geometry_gdf, how='left', predicate='within')
joined_data_county_gdf['lon'] = joined_data_county_gdf.geometry.x
joined_data_county_gdf['lat'] = joined_data_county_gdf.geometry.y

# encode counties
joined_data_county_gdf = pd.get_dummies(joined_data_county_gdf, columns=['COUNTY'], dtype=int)

# drop extraneous columns that likely have little predictive value based on intuition
joined_data_county_gdf = joined_data_county_gdf.drop(columns=['datetype', 'index_right', 'geometry', 'CaseIdentifier', '__id', 'death_month'])
joined_data_county_gdf = joined_data_county_gdf.drop(columns=['injuryplace',
                                                              'descriptionofinjury', 
                                                              'location', # location could be good to use in future - omitted for simplicity
                                                              'locationifother',
                                                              'mannerofdeath',
                                                              'othersignifican',
                                                              'cod',
                                                              'other'
                                                    ])

# we have latitude and longitude, as well as county, so these columns are redundant and likely correlated
column_drop_keywords = ['residence', 'state', 'county', 'city']
columns_to_drop = [col for col in joined_data_county_gdf.columns if any(keyword in col for keyword in column_drop_keywords) and 'ethnicity' not in col]
joined_data_county_gdf = joined_data_county_gdf.drop(columns=columns_to_drop)
print(joined_data_county_gdf.columns)

Index(['date', 'age', 'sex', 'race', 'ethnicity', 'heroin', 'heroin_dc',
       'cocaine', 'fentanyl', 'fentanylanalogue', 'oxycodone', 'oxymorphone',
       'ethanol', 'hydrocodone', 'benzodiazepine', 'methadone',
       'meth_amphetamine', 'amphet', 'tramad', 'hydromorphone',
       'morphine_notheroin', 'xylazine', 'gabapentin', 'opiatenos',
       'heroin_morph_codeine', 'other_opioid', 'anyopioid', 'death_day',
       'death_month_num', 'death_year', 'death_day_of_week',
       'death_day_is_weekend', 'death_day_week_of_year', 'sex_encoded', 'lon',
       'lat', 'COUNTY_Fairfield', 'COUNTY_Hartford', 'COUNTY_Litchfield',
       'COUNTY_Middlesex', 'COUNTY_New Haven', 'COUNTY_New London',
       'COUNTY_Tolland', 'COUNTY_Windham'],
      dtype='object')


In [18]:
# print(joined_data_county_gdf['injuryplace'].value_counts()) # this is mostly values like home and residence as top 2 most common values, then other is third which isn't helpful.
# print(joined_data_county_gdf['descriptionofinjury'].value_counts()) # this is mostly values like substance abuse, drug use, other descriptions we already know.
# print(joined_data_county_gdf['location'].value_counts()) # this looks usable, it is a good mix of Residence and Hospital. *NOTE: I still exclude this for simplicity.
# print(joined_data_county_gdf['mannerofdeath'].value_counts()) # this is almost all values of accident.
# print(joined_data_county_gdf['cod'].value_counts()) # there are too many free text variations, and they don't add new info to the discrete drug columns.


**Note**: In the future it would be interesting to apply methods to make use of free-text in certain columns. But for now, we are going to ignore them for simplicity and the assumption they don't add new information. For example, descriptionofinjury saying 'ingested prescription medication' doesn't add new information when we have columns for the presence of specific drugs.

Pre-process age by normalizing.

In [19]:
# from sklearn.preprocessing import MinMaxScaler
# age_scaler = MinMaxScaler()
# joined_data_county_gdf['age_normalized'] = age_scaler.fit_transform(joined_data_county_gdf[['age']])
# columns_received_encoding.append('age')

Pre-process ethnicity.

In [20]:
joined_data_county_gdf['ethnicity'].value_counts()

ethnicity
Hispanic                                        949
No, not Spanish/Hispanic/Latino                 723
Spanish/Hispanic/Latino                         261
Other Spanish/Hispanic/Latino                   240
Yes, other Spanish/Hispanic/Latino              189
Not Spanish/Hispanic/Latino                      50
Yes, Other Spanish/Hispanic/Latino (Specify)     24
Puerto Rican                                     19
Unknown                                          15
Yes, Puerto Rican                                10
Yes, Mexican, Mexican American, Chicano           6
Mexican, Mexican American, Chicano                4
Cuban                                             2
Name: count, dtype: int64

In [21]:
def classify_not_hispanic_latino(ethnicity):
    # this assumes all possible values for non-hispanic/latino contain 'not'
    if (not isinstance(ethnicity, str)) or ('not'.lower() in ethnicity.lower()):
        return 0
    else:
        return 1

In this dataset, all ethnicity values that represent a person not being hispanic/latino contain the word 'not', so the ethnicity column will be used to create a new hispanic/latino column (1 if yes, 0 if no)

In [22]:
joined_data_county_gdf['ethnicity_hispanic_latino'] = joined_data_county_gdf['ethnicity'].apply(classify_not_hispanic_latino)
columns_received_encoding.append('ethnicity')

Pre-process race column.

In [23]:
joined_data_county_gdf['race'].value_counts()

race
White                                                       9869
Black or African American                                    802
Black                                                        785
Unknown                                                       60
Other                                                         52
Asian Indian                                                  25
Asian, Other                                                  22
Other Asian                                                   12
Other (Specify)                                               12
Asian/Indian                                                   5
Asian                                                          2
white                                                          2
Chinese                                                        2
Korean                                                         1
American Indian or Alaska Native                               1
Native American, Oth

In [24]:
def consolidate_race(race):
    if not isinstance(race, str):
        return 'Other'
    if 'white'.lower() in race.lower():
        return 'White'
    if 'black'.lower() in race.lower():
        return 'Black'
    if ('asian'.lower() in race.lower()) or ('chinese'.lower() in race.lower()) or ('korean'.lower() in race.lower()):
        return 'Asian'
    else:
        return 'Other'

In [25]:
joined_data_county_gdf['race'] = joined_data_county_gdf['race'].apply(consolidate_race)

In [26]:
joined_data_county_gdf = pd.get_dummies(joined_data_county_gdf, columns=['race'], prefix='race', dtype=int)

Pre-process death_day_of_week column. I chose to use one-hot encoding to prevent models from assuming an implicit order between days. A disadvantage is that there is a loss of cyclic information.

**Note**: An alternative I considered was using cyclic encoding with sin and cos with a period of 7 steps.
e.g: sin_day = sin($2\pi$ * day_number/7), cos_day = cos($2\pi$ * day_number/7)

In [27]:
joined_data_county_gdf = pd.get_dummies(joined_data_county_gdf, columns=['death_day_of_week'], prefix='day_of_week', dtype=int)

Pre-process death_day_is_weekend column.

In [28]:
joined_data_county_gdf['death_day_is_weekend'] = joined_data_county_gdf['death_day_is_weekend'].astype(int)

Re-order feature and target columns.

In [29]:
first_target_col_index = joined_data_county_gdf.columns.get_loc('heroin')
last_target_col_index = joined_data_county_gdf.columns.get_loc('anyopioid')
target_cols = joined_data_county_gdf.columns[first_target_col_index:last_target_col_index+1]
target_df = joined_data_county_gdf[target_cols]
feature_df = joined_data_county_gdf.drop(columns=target_cols)

In [30]:
# target_df['other_opioid'].value_counts()
# this column is a mix of Y for yes and free text - let's remove this
target_df = target_df.drop(columns=['other_opioid'])
target_df = ~target_df.isnull()
target_df = target_df.astype(int)

In [32]:
# I did this in case we needed one df where features and targets were both present, but it doesn't seem to be needed
# ordered_col_data = pd.concat([feature_df, target_df], axis=1)
# ordered_col_data = ordered_col_data.drop(columns=columns_received_encoding)

In [33]:
reference_date = pd.Timestamp('2012-01-01')
feature_df['days_since_reference'] = (feature_df['date'] - reference_date).dt.days
columns_received_encoding.append('date')

In [34]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler()
columns_to_scale = ['age', 'lat', 'lon', 'death_year', 'days_since_reference']
feature_df[columns_to_scale] = minmax_scaler.fit_transform(feature_df[columns_to_scale])

In [35]:
# drop feature columns that aren't machine-readable
feature_df = feature_df.drop(columns=columns_received_encoding)
feature_df = feature_df.drop(columns=['death_day', 'death_month_num', 'death_day_week_of_year'])
feature_df = feature_df.fillna(0)

In [36]:
# print(f"total columns: {len(ordered_col_data.columns)}")
# numeric_df = ordered_col_data.select_dtypes(include=np.number)
# print(f"total numeric columns: {len(numeric_df.columns)}")

## Fit the Model <a class="anchor" id="fit-model"></a>

In [37]:
from sklearn.model_selection import train_test_split
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import hamming_loss, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix


X_train, X_test, Y_train, Y_test = train_test_split(feature_df.values, target_df.values, test_size=0.30, random_state=3008)

model = BinaryRelevance(classifier=MultinomialNB(), require_dense=[True, True])

model.fit(X_train, Y_train)

predictions = model.predict(X_test)
predicted_array = predictions.toarray()

hl = hamming_loss(Y_test, predicted_array)
print(f"Hamming Loss: {hl}")

ascore = accuracy_score(Y_test, predicted_array)
print(f"Accuracy Score: {ascore}")

Hamming Loss: 0.11544011544011544
Accuracy Score: 0.09376786735277301


These metrics are not good. Let's try a simpler classification model using just one target. I chose heroin as the target to predict.

## Single Target Classifier <a class="anchor" id="single-class"></a>

For our reference, the naive classifier that predicts the most common label will be used as the baseline

In [49]:
def calculate_score_metrics(Y_test, Y_predicted):   
    accuracy = accuracy_score(Y_test, Y_predicted) # correct / total
    precision = precision_score(Y_test, Y_predicted, zero_division=np.nan) # true positive / (true + false positive).       aka correctly classified actual positives / everything classified as positive
    recall = recall_score(Y_test, Y_predicted) # true positive / (true positive + false negative).    aka correctly classified actual positives / all actual positives
    f1 = f1_score(Y_test, Y_predicted) # 2 * Precision * Recall / (Precision + Recall).               aka harmonic mean of precision and recall
    roc_auc = roc_auc_score(Y_test, Y_predicted) # how well classes are separated

    print(f"Accuracy Score: {accuracy}")
    print(f"Precision Score: {precision}")
    print(f"Recall Score: {recall}")
    print(f"F1 Score: {f1}")
    print(f"ROC AUC Score: {roc_auc}")

In [51]:
heroin_labels = target_df['heroin']

X_train, X_test, Y_train, Y_test = train_test_split(feature_df.values, heroin_labels.values, test_size=0.30, random_state=3008)

most_common_heroin_label = heroin_labels.mode()[0]
naive_predictions = np.full(Y_test.size, most_common_heroin_label)
print("Naive Classifier")
calculate_score_metrics(Y_test, naive_predictions)

Naive Classifier
Accuracy Score: 0.7009719839908519
Precision Score: nan
Recall Score: 0.0
F1 Score: 0.0
ROC AUC Score: 0.5


In [52]:
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(random_state=3009)
random_forest_model.fit(X_train, Y_train)

rf_pred = random_forest_model.predict(X_test)

print("Random Forest Classifier for heroin as cause of death")
calculate_score_metrics(Y_test, rf_pred)

Random Forest Classifier for heroin as cause of death
Accuracy Score: 0.73642081189251
Precision Score: 0.5759803921568627
Recall Score: 0.44933078393881454
F1 Score: 0.5048335123523093
ROC AUC Score: 0.6541107427035019


In [53]:
from sklearn.neighbors import KNeighborsClassifier
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, Y_train)

knn_predicted = knn_model.predict(X_test)

print("KNN Classifier (N=5) for heroin as cause of death")
calculate_score_metrics(Y_test, knn_predicted)

KNN Classifier (N=5) for heroin as cause of death
Accuracy Score: 0.710691823899371
Precision Score: 0.5213032581453634
Recall Score: 0.3977055449330784
F1 Score: 0.4511930585683297
ROC AUC Score: 0.620957177034239


In [55]:
from sklearn.linear_model import LogisticRegression

logistic_regression_model = LogisticRegression()
logistic_regression_model.fit(X_train, Y_train)

log_reg_predicted = logistic_regression_model.predict(X_test)

print("Logistic Regression Classifier for heroin as cause of death")
calculate_score_metrics(Y_test, log_reg_predicted)

Logistic Regression Classifier for heroin as cause of death
Accuracy Score: 0.7198399085191538
Precision Score: 0.5522151898734177
Recall Score: 0.33365200764818354
F1 Score: 0.4159713945172825
ROC AUC Score: 0.6091180103493772
